<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/files_2020/cce-color.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Pipeline de Datos de Contratación Pública
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>

## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Septiembre 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Pipeline de Datos de Contratación Pública**|
|**Descripción y alcance**|Notebook para la limpieza y validación de los datos de la base unificada.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos - GAEC</td></tr><tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Maria del Pilar Suarez Sebastian</td></tr> <tr><td>*Cargo:*</td> <td>Subdirectora Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

# Introducción

En este notebook, nos centraremos en la limpieza y preprocesamiento de los datos obtenidos de la base de contratos del Sistema Electrónico para la Contratación Pública (SECOP) I y II de Colombia. Los conjuntos de datos con los que estamos trabajando contienen una variedad de información sobre contratos públicos, desde sus detalles operativos hasta sus atributos financieros.

El objetivo principal de este ejercicio es preparar estos conjuntos de datos para análisis posteriores. Esta preparación incluye una serie de pasos que asegurarán que los datos estén en el formato correcto y sean consistentes, comprensibles y confiables.

Cargamos la base previamente descargada y con las variables unificadas entre los conjuntos de datos de cada fuente para asegurar que la misma información se represente de manera coherente en los mismos. Esto puede incluir el mapeo de categorías equivalentes entre los conjuntos de datos y la armonización entre variables y sus nombres.

A continuación, estandarizaremos los datos para garantizar que todas las variables se presenten en un formato uniforme. Este proceso puede involucrar la normalización de texto, la conversión de datos categóricos en formatos numéricos y la manipulación de fechas y tiempos para que se presenten de manera consistente.

Una parte crucial de este trabajo implica limpiar los datos de elementos no deseados. Esto implica quitar espacios extra, eliminar caracteres especiales y, en general, asegurarnos de que los datos sean lo más limpios y precisos posible.

Además, verificaremos los tipos de variables en nuestros conjuntos de datos para asegurarnos de que sean apropiados para el tipo de datos que contienen. Esto podría implicar la conversión de datos numéricos almacenados como texto en números reales, o viceversa.

Comprobaremos si hay registros duplicados en nuestros conjuntos de datos. Si existen, los almacenaremos para realizar la validación y evitar cualquier sesgo o inexactitud en los análisis posteriores.

Finalmente, validaremos la cantidad de valores nulos y los valores negativos en la variable de cantidad de contrato. En el caso de los valores nulos, decidiremos si tienen coherencia, dependiendo del contexto. Los valores negativos, si no tienen sentido en el contexto de nuestro análisis, serán almacenados y validados desde la descarga de los datos.

Al final de este proceso, tendremos un conjunto de datos limpio y de alta calidad que estará listo para ser utilizado en futuros análisis y modelado de datos.

## Cargue del archivo parquet

### Cargue de librerías

En esta sección se cargan las librerías necesarias para el desarrollo del script y se establecen los parámetros de trabajo.

In [1]:
### Paquetes usados para la exploración de datos

import re
import pandas as pd
import numpy as np
# import matplotlib.pyplot as pltk
import datetime as dt
# import unidecode
# from dataprep.clean import clean_duplication
import warnings
warnings.filterwarnings('ignore')

# Librerías creadas para la construcción del maestro
from libreria import data_management as dm
from libreria import entity_management as em

In [3]:
# years = ['2020', '2021', '2022', '2023']
years = ['2023']
SECOP = {}

for year in years:
    SECOP[year] = pd.read_csv(f'../../../muestras de datos/procesados/bronce/SECOP_{year}.csv', sep=';')
# for year in years:
#     SECOP[year] = pd.read_csv(f'../../../muestras de datos/procesados/silver/SECOP_{year}_pre.csv', sep=';')

In [4]:
SECOP[year].shape

(1307189, 27)

## Actualización del maestro de entidades 

In [5]:
# Se carga el maestro de entidades
entity_master = pd.read_excel(f'../../../muestras de datos/procesados/silver/maestro_entidades.xlsx')

for year in years:
    SECOP[year] = SECOP[year].rename(columns={'ID_Entidad': 'ID_SECOP', 'Nombre_Entidad': 'ENTIDAD', 'NIT_Entidad': 'NIT', 'Orden_Entidad': 'ORDEN', 
                                              'Departamento_Entidad': 'DEPARTAMENTO', 'Municipio_Entidad': 'MUNICIPIO'})
    
    # Se separan las entidades que no tienen nombre de las que sí para evitar registros nulos sobre el maestro de entidades
    con_entidad = SECOP[year][~(SECOP[year]['ENTIDAD'].isna())]
    sin_entidad = SECOP[year][SECOP[year]['ENTIDAD'].isna()]

    con_entidad['ENTIDAD'] = con_entidad['ENTIDAD'].astype(str)
    con_entidad['ENTIDAD'] = con_entidad['ENTIDAD'].apply(dm.remove_extra_punct)
    # Se quitan los digitos de verificacion que se pueden identificar por el guion
    con_entidad['NIT'] = con_entidad['NIT'].str.extract(r'(\d+)-?\d*')
    # Se relacionan las entidades de los contratos y se actualiza simultaneamente el maestro de entidades
    con_entidad = em.update_entity_master(con_entidad, entity_master)
    # Se eliminan las columnas innecesarias sobre el maestro de contratos ya que los registros se podrán relacionar con el maestro de entidades por el campo ID_ENTIDAD
    con_entidad = con_entidad.drop(columns=['ID_SECOP', 'ENTIDAD', 'NIT', 'ORDEN', 'DEPARTAMENTO', 'MUNICIPIO'])
    sin_entidad = sin_entidad.drop(columns=['ID_SECOP', 'ENTIDAD', 'NIT', 'ORDEN', 'DEPARTAMENTO', 'MUNICIPIO'])

    con_entidad = con_entidad.rename(columns={'ID': 'ID_ENTIDAD'})
    sin_entidad['ID_ENTIDAD'] = -1 # Se marcan con -1 los ID_ENTIDAD de los contratos que no tienen entidad registrada

    # Se guardan los registros en el maestro de contratos
    SECOP[year] = pd.concat([con_entidad, sin_entidad])

In [6]:
for year in years:
    SECOP[year].to_csv(f'../../../muestras de datos/procesados/silver/SECOP_{year}_pre.csv', sep=';')

Validación tipos de variables

In [7]:
def clean_dates(date):
    if pd.isna(date):
        return None  # Si la fecha es nula, devuelve nulo
    try:
        return pd.to_datetime(date, errors='ignore').strftime('%d-%m-%Y')
    except (ValueError, TypeError):
        return None

In [8]:
for year in years:
    SECOP[year]['ID_Contrato'] = SECOP[year]['ID_Contrato'].astype(str)
    SECOP[year]['ID_Proceso'] = SECOP[year]['ID_Proceso'].astype(str)
    SECOP[year]['Modalidad'] = SECOP[year]['Modalidad'].astype(str)
    SECOP[year]['Estado'] = SECOP[year]['Estado'].astype(str)
    SECOP[year]['Descripcion_proceso'] = SECOP[year]['Descripcion_proceso'].astype(str)
    SECOP[year]['Objeto_Contrato'] = SECOP[year]['Objeto_Contrato'].astype(str)
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].astype(str)
    SECOP[year]['Nombre_Proveedor'] = SECOP[year]['Nombre_Proveedor'].astype(str)
    SECOP[year]['Documento_Proveedor'] = SECOP[year]['Documento_Proveedor'].astype(str)
    SECOP[year]['Tipo_proveedor'] = SECOP[year]['Tipo_proveedor'].astype(str)
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].astype(str)
    SECOP[year]['Municipio_Proveedor'] = SECOP[year]['Municipio_Proveedor'].astype(str)
    SECOP[year]['Link'] = SECOP[year]['Link'].astype(str)

    SECOP[year]['Valor_contrato'] = SECOP[year]['Valor_contrato'].astype(float)

    SECOP[year]['Fecha_firma'] = pd.to_datetime(SECOP[year]['Fecha_firma'], format='mixed')
    # SECOP[year]['Fecha_inicio_contrato'] = SECOP[year]['Fecha_inicio_contrato'].apply(clean_dates)
    # SECOP[year]['Fecha_fin_contrato'] = SECOP[year]['Fecha_fin_contrato'].apply(clean_dates)

In [9]:
def replace_nan(value):
    if value == 'nan':
        return ''
    return value

In [10]:
for year in years:
    SECOP[year]['ID_Contrato'] = SECOP[year]['ID_Contrato'].apply(replace_nan)
    SECOP[year]['ID_Proceso'] = SECOP[year]['ID_Proceso'].apply(replace_nan)
    SECOP[year]['Modalidad'] = SECOP[year]['Modalidad'].apply(replace_nan)
    SECOP[year]['Estado'] = SECOP[year]['Estado'].apply(replace_nan)
    SECOP[year]['Descripcion_proceso'] = SECOP[year]['Descripcion_proceso'].apply(replace_nan)
    SECOP[year]['Objeto_Contrato'] = SECOP[year]['Objeto_Contrato'].apply(replace_nan)
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].apply(replace_nan)
    SECOP[year]['Nombre_Proveedor'] = SECOP[year]['Nombre_Proveedor'].apply(replace_nan)
    SECOP[year]['Documento_Proveedor'] = SECOP[year]['Documento_Proveedor'].apply(replace_nan)
    SECOP[year]['Tipo_proveedor'] = SECOP[year]['Tipo_proveedor'].apply(replace_nan)
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].apply(replace_nan)
    SECOP[year]['Municipio_Proveedor'] = SECOP[year]['Municipio_Proveedor'].apply(replace_nan)
    SECOP[year]['Link'] = SECOP[year]['Link'].apply(replace_nan)

Tratamiento de espacios y caracteres especiales

In [11]:
for year in years:
    SECOP[year]['Modalidad'] = SECOP[year]['Modalidad'].apply(dm.remove_extra_punct)
    SECOP[year]['Estado'] = SECOP[year]['Estado'].apply(dm.remove_extra_punct)
    SECOP[year]['Descripcion_proceso'] = SECOP[year]['Descripcion_proceso'].apply(dm.remove_extra_punct)
    SECOP[year]['Objeto_Contrato'] = SECOP[year]['Objeto_Contrato'].apply(dm.remove_extra_punct)
    SECOP[year]['Nombre_Proveedor'] = SECOP[year]['Nombre_Proveedor'].apply(dm.remove_extra_punct) 
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].apply(dm.remove_extra_punct)
    SECOP[year]['Municipio_Proveedor'] = SECOP[year]['Municipio_Proveedor'].apply(dm.remove_extra_punct)

Tratamiento de espacios con strip

In [12]:
for year in years:
    SECOP[year]['ID_Contrato'] = SECOP[year]['ID_Contrato'].str.strip()
    SECOP[year]['ID_Proceso'] = SECOP[year]['ID_Proceso'].str.strip()
    SECOP[year]['Modalidad'] = SECOP[year]['Modalidad'].str.strip()
    SECOP[year]['Estado'] = SECOP[year]['Estado'].str.strip()
    SECOP[year]['Descripcion_proceso'] = SECOP[year]['Descripcion_proceso'].str.strip()
    SECOP[year]['Objeto_Contrato'] = SECOP[year]['Objeto_Contrato'].str.strip()
    SECOP[year]['Nombre_Proveedor'] = SECOP[year]['Nombre_Proveedor'].str.strip()
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].str.strip()
    SECOP[year]['Municipio_Proveedor'] = SECOP[year]['Municipio_Proveedor'].str.strip()

Estandarización de las modalidades

In [13]:
for year in years:
    SECOP[year]["Modalidad Estandar"] = SECOP[year]['Modalidad'].apply(dm.Tipo_Proceso)
    SECOP[year]["Grupo Modalidad"] = SECOP[year]["Modalidad Estandar"].apply(dm.Grupo_Modalidad)

Arreglo sobre estado

In [14]:
estado_contractual = {
    'modificado': 'Modificado',
    'celebrado': 'Celebrado',
    'terminado': 'Terminado',
    'liquidado': 'Liquidado',
    'cerrado': 'Cerrado',
    'activo': 'Activo',
    'cedido': 'Cedido',
    'suspendido': 'Suspendido',
    'convocado': 'Convocado',
    'terminado liquidar': 'Terminado liquidar',
    'terminado anormalmente despues convocado': 'Terminado anormalmente despues convocado',
    'terminado': 'Terminado',
    'ejecución': 'En ejecución',
    'cancelado': 'Cancelado',
    'enviado proveedor': 'Enviado proveedor'
}

for year in years:
    SECOP[year]['Estado'] = SECOP[year]['Estado'].replace(estado_contractual)

In [15]:
SECOP[year]['Estado'].value_counts()

Estado
Celebrado             393023
Liquidado              99097
Terminado liquidar     13877
                        9604
ejecucion               9190
Modificado              8575
Activo                  2038
Terminado               1334
Convocado                654
Cerrado                  347
adjudicado               209
Cedido                   132
Suspendido                49
Cancelado                  2
Name: count, dtype: int64

Arreglo sobre UNSPSC:

1. Separación por segmento, familia, clase y producto

In [16]:
for year in years:
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].str.replace('V1', '')
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].str.replace('.0', '')
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].str.replace('UNSPECIFIED', '')
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].str.replace('.', '')
    if SECOP[year]['UNSPSC'].str != '':
        SECOP[year]['SEGMENTO_UNSPSC'] = SECOP[year]['UNSPSC'].str[:2]
        SECOP[year]['FAMILIA_UNSPSC'] = SECOP[year]['UNSPSC'].str[2:4]
        SECOP[year]['CLASE_UNSPSC'] = SECOP[year]['UNSPSC'].str[4:6]
        SECOP[year]['PRODUCTO_UNSPSC'] = SECOP[year]['UNSPSC'].str[6:]

Arreglo sobre Departamento_Entidad y Departamento_Proveedor

In [17]:
for year in years:
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].str.replace('bogota d.c .', 'bogota d.c.')
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].str.replace('distrito capital bogota', 'bogota d.c.')

In [18]:
for year in years:
    SECOP[year].to_csv(f'../../../muestras de datos/procesados/silver/SECOP_{year}.csv', index=False, sep=';')

In [19]:
SECOP[year].shape

(538131, 28)

# Cruce con RUES

In [19]:
df_pnp = pd.read_csv('../../../muestras de datos/sin procesar/ReporteRUES.txt',sep="|", encoding="utf-8", dtype={'CIIU Principal': str})
df_pnp.shape

(3598233, 70)

In [20]:
df_pnp_1=df_pnp[df_pnp["Cod Estado Matricula"]==1]
df_pnp_1.shape

(3596567, 70)

***Observación***: Quedamos con un total de 3'596.567 de registros en la base, de 3'598.233 inicialmente. Lo que nos dice que hemos quitado 1.666 registros. 

Posteriormente vamos a verificar los `códigos de tamaño` ya que es la información que deseamos tener en la base de contratos del `SECOP` y aparecen algunos con ese campo ***nulo.***

In [21]:
df_pnp_2 = df_pnp_1[~df_pnp_1['CODIGO_TAMANO_EMPRESA'].isnull()]
df_pnp_2.shape

(3534150, 70)

***Observación:*** Obtenemos un total de 3'534.150 registros, realizando esta depuración. Se quitan 62.417 registros.

Procedemos a realizar el procesamiento grueso, en sentido de la limpieza de los `NITS`, este proceso nos arrojará la base final de **RUES** a utilizar para los cruces posteriores.

In [22]:
df_pnp_2['Numero de Identificación'] = df_pnp_2['Numero de Identificación'].astype(str)
df_pnp_2['NIT']=df_pnp_2['Numero de Identificación'].apply(lambda x: re.sub('[^0-9]','',x))
df_pnp_2=df_pnp_2[~df_pnp_2['NIT'].str.contains('00000000|99999999|111111111|222222222|444444444|33333333|55555555|66666666|77777777|88888888')]

df_pnp_2.shape

(3504158, 71)

***Observación:*** Cerca de 30 mil registros se quitan realizando esta limpieza, dejando una base de 3'504.158. Se va ahora a eliminar los registros según las longitudes de los mismos.

In [23]:
df_pnp_2 = df_pnp_2[(df_pnp_2['NIT'].apply(len)<=13) & (df_pnp_2['NIT'].apply(len)>=5)]
df_pnp_2.shape

(3402130, 71)

***Observación:*** Más de 100 mil NITS se eliminan en este paso, dejando una base de 3'402.130. 

***Observaciones***: Revisando los `NITS` que se repiten, se evidencia que son empresas distintas pero para el efecto del cruce a lo cual se le quita el código de verificación se trae la información del tamaño y revisando en la mayoría el __tamaño__ sale con el valor de 0, es decir _No definido_. Se procede a eliminar duplicados y dejar la base de **RUES** a este nivel de depuración.

In [24]:
df_pnp_2.drop_duplicates(subset=['NIT'],keep='last',inplace=True)
df_pnp_2.shape

(3396437, 71)

In [25]:
df_pnp_2 = df_pnp_2[['NIT','CIIU Principal','CIIU Secundario', 'CIIU 3', 'CIIU 4', 'ciiu_mayores_ingresos','Cód Cámara', 'Nombre de la Cámara de Comercio',
                     'Código Organización Jurídica', 'Descripción de la Organización Jurídica', 'Código del tipo de sociedad', 
                     'Descripción del Tipo de Sociedad', 'Código Categoría', 'Descripción de la Categoría', 'Cod Estado Matricula', 'Municipio Comercial', 
                     'Departamento Comercial','CODIGO_TAMANO_EMPRESA', 'Correo Electronico Comercial', 'Correo Electronico Fiscal','Número de Empleados', 
                     'Genero', 'Cantidad de Mujeres Empleadas', 'Cod Clase Identificación','Razón Social']]

***Observaciones:*** Eliminando los `NITS` duplicados para evitar múltiples cruces, nos da una base de __RUES__ depurada con 3'396.437 registros.

División de datos del SECOP para cruces con RUES

In [26]:
for year in years:
    SECOP[year]['NEW_NIT'] = SECOP[year]['Documento_Proveedor'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
    SECOP[year]['Tamaño_nit']=SECOP[year]['NEW_NIT'].apply(len)

In [27]:
SECOP[year]['Estado'].value_counts()

Estado
                      9603
Celebrado             3641
ejecucion              758
Liquidado              652
Modificado             216
Activo                 210
Terminado liquidar     109
Terminado               44
Cerrado                 43
Cedido                   9
Convocado                8
Suspendido               3
Name: count, dtype: int64

In [28]:
SECOP_RUES = {}
SECOP_NO_RUES = {}

condiciones_para_cruce_rues = ( 
    (SECOP[year]['Documento_Proveedor'] != 'NO DEFINIDO') &
    (SECOP[year]['Estado'] != 'Enviado proveedor') &
    (SECOP[year]['Valor_contrato'] <= 20000000000000) &
    (SECOP[year]['Tamaño_nit'] >= 5) &
    (SECOP[year]['Tamaño_nit'] <= 13)
)

for year in years:
    SECOP_RUES[year] = SECOP[year][condiciones_para_cruce_rues]
    SECOP_NO_RUES[year] = SECOP[year][~condiciones_para_cruce_rues]

In [29]:
CRUCE_RUES = {}

for year in years:
    CRUCE_RUES[year] = pd.merge(SECOP_RUES[year], df_pnp_2, how='left', left_on='NEW_NIT', right_on='NIT', indicator=True)
    no_cruzaron = CRUCE_RUES[year][CRUCE_RUES[year]['_merge'] == 'left_only']
    CRUCE_RUES[year] = CRUCE_RUES[year][CRUCE_RUES[year]['_merge'] == 'both']
    no_cruzaron = no_cruzaron.drop(columns=['NIT','CIIU Principal','CIIU Secundario', 'CIIU 3', 'CIIU 4', 'ciiu_mayores_ingresos','Cód Cámara', 
                                            'Nombre de la Cámara de Comercio', 'Código Organización Jurídica', 'Descripción de la Organización Jurídica', 
                                            'Código del tipo de sociedad', 'Descripción del Tipo de Sociedad', 'Código Categoría', 'Descripción de la Categoría', 
                                            'Cod Estado Matricula', 'Municipio Comercial', 'Departamento Comercial','CODIGO_TAMANO_EMPRESA', 
                                            'Correo Electronico Comercial', 'Correo Electronico Fiscal','Número de Empleados', 'Genero', 
                                            'Cantidad de Mujeres Empleadas', 'Cod Clase Identificación','Razón Social', '_merge'])
    CRUCE_RUES[year] = CRUCE_RUES[year].drop(columns='_merge')
    no_cruzaron['NIT2'] = no_cruzaron['NEW_NIT'].apply(lambda x: x[:-1])
    cruce2 = pd.merge(no_cruzaron, df_pnp_2, how='left',left_on='NIT2',right_on='NIT')
    CRUCE_RUES[year] = pd.concat([CRUCE_RUES[year], cruce2])


In [30]:
for year in years:
    SECOP[year] = pd.concat([CRUCE_RUES[year], SECOP_NO_RUES[year]])

### Próximos requerimientos

- Cruce con la versión anterior del maestro de entidades (campos ```ID_Entidad, Nombre_Entidad, NIT_Entidad```)
- Actualización del maestro de entidades
- Limpieza del campo NIT_Entidad (digito de verificación)
- Arreglo sobre Documento_Proveedor
- Cruce de departamentos y municipios con el código divipola

### Validaciones sobre la versión actual

* Verificar duplicados en contratos

In [31]:
SECOP[year]['ID_Contrato'].value_counts().reset_index().head(10)

,ID_Contrato,count
0,,9603
1,12694007,11
2,12537403,8
3,12537267,7
4,12548585,6
5,12803479,6
6,12548081,6
7,12544510,5
8,12658111,4
9,12693057,4


In [32]:
duplicados_2023 = SECOP['2023']['ID_Contrato'].value_counts().reset_index()

In [33]:
print('Cantidad de contratos duplicados:')
print(len(duplicados_2023[(duplicados_2023['ID_Contrato']>1)]))
duplicados_2023[(duplicados_2023['ID_Contrato']>1)].head(10)

Cantidad de contratos duplicados:


TypeError: '>' not supported between instances of 'str' and 'int'

Detectar negativos en valor del contrato

In [ ]:
for year in years:
    print('Cantidad de contratos con valor negativo 2023:', len(SECOP['2023'][(SECOP['2023']['Valor_contrato']<0)]))
    print('ID de los contratos 2023:', SECOP['2023'][(SECOP['2023']['Valor_contrato']<0)]['ID_Contrato'].unique())

Se unifican las varibles departamento_entidad, municipio_entidad, departamento_proveedor y municipio_proveedor

In [ ]:
columnas_geograficas = ['Departamento_Entidad','Departamento_Proveedor', 'Municipio_Entidad', 'Municipio_Proveedor']

for column in columnas_geograficas:
    for year in years:
        SECOP[year][column] = SECOP[year][column].str.upper()
        # Se llenan los campos vacíos con "No Definido"
        SECOP[year][column] = SECOP[year][column].fillna("No Definido")

Se ven los primeros registros del dataframe

In [ ]:
SECOP['2023'].head(5)

# Cruce con el maestro de entidades

In [ ]:
import pandas as pd

In [ ]:
years = ['2020', '2021', '2022', '2023']
SECOP = {}

for year in years:
    SECOP[year] = pd.read_csv(f'../../../muestras de datos/procesados/silver/SECOP_{year}.csv', sep=';')

In [ ]:
mdm_entidades = pd.read_excel('../../../muestras de datos/auxiliar/20220714_MDM_Entidades_v6.xlsx')
mdm_entidades = mdm_entidades.drop(columns=['codigo_entidad', 'codigo_sigep', 'codigo_sigep.1', 'Fuente', 'Municipio', 'NOMBRE_DPT',
                                            'orden_Entidad', 'rama_Entidad', 'sector_Entidad', 'llave', 'Id_Ciudad', 'Obligada SECOP II', 'Entidad Ciudad Capital',
                                            'TVEC', 'validacion_nombre', 'Año_Obligatoriedad', 'Mes_Ingreso', 'Trimestre_Ingreso'])

In [ ]:
mdm_entidades.columns

In [ ]:
SECOP['2023'].columns

In [ ]:
mdm_entidades['NIT_Secop'] = mdm_entidades['NIT_Secop'].astype(str)
mdm_entidades['N_Entidad_Plataforma'] = mdm_entidades['N_Entidad_Plataforma'].astype(str)
mdm_entidades['N_Entidad_Final'] = mdm_entidades['N_Entidad_Final'].astype(str)
mdm_entidades['Nombre Estandar'] = mdm_entidades['Nombre Estandar'].astype(str)
mdm_entidades['N_Entidad_FP'] = mdm_entidades['N_Entidad_FP'].astype(str)

mdm_entidades['N_Entidad_Plataforma'] = mdm_entidades['N_Entidad_Plataforma'].apply(dm.remove_extra_punct)
mdm_entidades['N_Entidad_Final'] = mdm_entidades['N_Entidad_Final'].apply(dm.remove_extra_punct)
mdm_entidades['Nombre Estandar'] = mdm_entidades['Nombre Estandar'].apply(dm.remove_extra_punct)
mdm_entidades['N_Entidad_FP'] = mdm_entidades['N_Entidad_FP'].apply(dm.remove_extra_punct)

In [ ]:
for year in years:
    mdm_entidades_unique = mdm_entidades.drop_duplicates(subset='N_Entidad_Final')
    cruce_nom_entidad1 = pd.merge(SECOP[year], mdm_entidades_unique, left_on='Nombre_Entidad', right_on='N_Entidad_Final', how='left')
    cruce_nom_entidad1 = cruce_nom_entidad1[cruce_nom_entidad1['NIT_Secop'].notnull()]
    mipymes_sin_cruce = SECOP[year][~SECOP[year]['Nombre_Entidad'].isin(mdm_entidades_unique['N_Entidad_Final'])]

    mdm_entidades_unique = mdm_entidades.drop_duplicates(subset='N_Entidad_Plataforma')
    cruce_nom_entidad2 = pd.merge(mipymes_sin_cruce, mdm_entidades_unique, left_on='Nombre_Entidad', right_on='N_Entidad_Plataforma', how='left')
    cruce_nom_entidad2 = cruce_nom_entidad2[cruce_nom_entidad2['NIT_Secop'].notnull()]
    mipymes_sin_cruce = mipymes_sin_cruce[~mipymes_sin_cruce['Nombre_Entidad'].isin(mdm_entidades_unique['N_Entidad_Plataforma'])]

    mdm_entidades_unique = mdm_entidades.drop_duplicates(subset='Nombre Estandar')
    cruce_nom_entidad3 = pd.merge(mipymes_sin_cruce, mdm_entidades_unique, left_on='Nombre_Entidad', right_on='Nombre Estandar', how='left')
    cruce_nom_entidad3 = cruce_nom_entidad3[cruce_nom_entidad3['NIT_Secop'].notnull()]
    mipymes_sin_cruce = mipymes_sin_cruce[~mipymes_sin_cruce['Nombre_Entidad'].isin(mdm_entidades_unique['Nombre Estandar'])]

    mdm_entidades_unique = mdm_entidades.drop_duplicates(subset='N_Entidad_FP')
    cruce_nom_entidad4 = pd.merge(mipymes_sin_cruce, mdm_entidades_unique, left_on='Nombre_Entidad', right_on='N_Entidad_FP', how='left')
    cruce_nom_entidad4 = cruce_nom_entidad4[cruce_nom_entidad4['NIT_Secop'].notnull()]
    mipymes_sin_cruce = mipymes_sin_cruce[~mipymes_sin_cruce['Nombre_Entidad'].isin(mdm_entidades_unique['N_Entidad_FP'])]

    mdm_entidades_unique = mdm_entidades.drop_duplicates(subset='NIT_Secop')
    cruce_nit_entidad = pd.merge(mipymes_sin_cruce, mdm_entidades_unique, left_on='NIT_Entidad', right_on='NIT_Secop', how='left')
    cruce_nit_entidad = cruce_nit_entidad.dropna(subset=['NIT_Secop'])
    mipymes_sin_cruce = mipymes_sin_cruce[~mipymes_sin_cruce['NIT_Entidad'].isin(mdm_entidades_unique['NIT_Secop'])]

    SECOP[year] = pd.concat([cruce_nit_entidad, cruce_nom_entidad1, cruce_nom_entidad2, cruce_nom_entidad3, cruce_nom_entidad4, mipymes_sin_cruce])

In [ ]:
SECOP['2023'].columns

In [ ]:
SECOP = pd.concat([SECOP['2020'], SECOP['2021'], SECOP['2022'], SECOP['2023']])
SECOP.shape

In [ ]:
len(SECOP[SECOP['Nombre Estandar'].isna()]['Nombre_Entidad'].unique())

In [ ]:
len(SECOP[~SECOP['Nombre Estandar'].isna()]['Nombre_Entidad'].unique()) + len(SECOP[SECOP['Nombre Estandar'].isna()]['Nombre_Entidad'].unique())

In [ ]:
1-(2073/13376)

In [ ]:
maestro_entidades = SECOP[~SECOP['Nombre Estandar'].isna()][['NIT_Entidad', 'Nombre_Entidad', 'Departamento_Entidad',
       'Municipio_Entidad', 'NIT_Secop', 'N_Entidad_Final', 'N_Entidad_Plataforma', 'Nombre Estandar']]

maestro_entidades.head(2)

Almacenamiento del archivo como CSV y como parquet

In [34]:
for year in years:
    SECOP[year].to_csv(f'../../../muestras de datos/procesados/silver/SECOP_{year}.csv', index=False, sep=';')

In [ ]:
# Se revisa cantidad de nulos de cada variable

# SECOP['2023'].isnull().sum()

# Conclusiones

Se realiza la limpieza de la base maestra, logrando el objetivo de tener un conjunto de datos limpio y de alta calidad que estará listo para ser utilizado en futuros análisis.

Se debe tener presente la alerta de valores duplicados en ID_Contrato, valores negativos en valor del contrato y valores nulos en variables donde se debe contar con la información (si los hay) para realizar las validaciones correspondientes.

El proceso se realiza con archivos descargados por año para agilizar la ejecución del notebook.

Finalmente, es importante recordar que la limpieza y el preprocesamiento de datos es un paso fundamental en cualquier proyecto de análisis de datos. La atención al detalle en esta etapa puede tener un impacto significativo en la calidad y la precisión de cualquier insight o modelo que se desarrolle en las etapas posteriores.